In [14]:
# this will merge your custom dataset into COCO for retraining.

In [2]:
#!/usr/bin/env python3
import datetime
import json
import os
import re
import fnmatch
from PIL import Image
import numpy as np
import sys
sys.path.append('/app/pycococreator/')
from pycococreatortools import pycococreatortools

WMT_SRC_DIR = '/data/productgraph_from-hive_non-lifestyle/'
DST_DIR = '/data/COCO_2017/train2017/'
SRC_ANN_FILE = '/data/COCO_2017/annotations/instances_train2017.json'
MERGED_ANN_FILE = '/data/COCO_2017/annotations/instances_merged.json'
# being lazy and hard coding this for now
#Area_Rugs  Bookcases  Desks  Dressers  Lamps  Nightstands  Stools  Television_Stands

categories = [
    {
        'id': 91,
        'name': 'Area_Rugs',
        'supercategory': 'furniture',
    },
        {
        'id': 92,
        'name': 'Bookcases',
        'supercategory': 'furniture',
    },
        {
        'id': 93,
        'name': 'Desks',
        'supercategory': 'furniture',
    },
        {
        'id': 95,
        'name': 'Dressers',
        'supercategory': 'furniture',
    },
        {
        'id': 96,
        'name': 'Lamps',
        'supercategory': 'furniture',
    },
        {
        'id': 97,
        'name': 'Nightstands',
        'supercategory': 'furniture',
    },
        {
        'id': 98,
        'name': 'Stools',
        'supercategory': 'furniture',
    },
        {
        'id': 99,
        'name': 'Television_Stands',
        'supercategory': 'furniture',
    }
]

In [3]:
def filter_for_jpeg(root, files):
    file_types = ['*.jpeg']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    return files

def filter_for_annotations(root, files, image_filename):
    file_types = ['*.png']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    basename_no_extension = os.path.splitext(os.path.basename(image_filename))[0]
    file_name_prefix = basename_no_extension + '.*'
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    files = [f for f in files if re.match(file_name_prefix, os.path.splitext(os.path.basename(f))[0])]
    return files

In [23]:
coco_anns = json.load(open(SRC_ANN_FILE))

In [24]:
import heapq
ann_ids = [x['id'] for x in coco_anns['annotations']]
heapq.heapify(ann_ids)
max_ann_id = heapq.nlargest(1, ann_ids)[0]
img_ids = [x['id'] for x in coco_anns['images']]
heapq.heapify(img_ids)
max_img_id = heapq.nlargest(1, img_ids)[0]
print("max_img_id {} max_ann_id {}".format(max_img_id, max_ann_id))
coco_anns.keys()
coco_anns['categories'] = coco_anns['categories'] + categories

max_img_id 581929 max_ann_id 909000554145


In [25]:
image_id = max_img_id + 1
# I am not sure how the segmentation id is helpful (or) used
# Need to dig into the documentation
segmentation_id = max_ann_id + 1
for root, _, files in os.walk(WMT_SRC_DIR):
    print(root)
    image_files = filter_for_jpeg(root, files)
    for fl in image_files:
        image = Image.open(fl)
        cgt = fl.split("/")[-2]
        flnme = fl.split("/")[-1]
        try:
            os.symlink(fl, "{}/{}".format(DST_DIR, flnme))
        except FileExistsError:
            pass
        image_info = pycococreatortools.create_image_info(image_id, os.path.basename(fl), image.size)
        coco_anns["images"].append(image_info)
        #print(image_info)
        ann_fl = re.sub('.jpeg', '-mask.png', fl)
        #print(ann_fl)
        class_id = [x['id'] for x in categories if x['name'] in cgt][0]
        category_info = {'id': class_id, 'is_crowd': 0}
        try:
            binary_mask = np.asarray(Image.open(ann_fl))
        except FileNotFoundError:
            continue
        #print(category_info, class_id)
        annotation_info = pycococreatortools.create_annotation_info(
            segmentation_id, image_id, category_info, binary_mask,
            tolerance=2)
            #image.size, tolerance=2)
        #print(pycococreatortools.binary_mask_to_polygon(binary_mask))
        #print(annotation_info)
        if annotation_info is not None:
            coco_anns["annotations"].append(annotation_info)
        segmentation_id = segmentation_id + 1
        image_id = image_id + 1
        
with open(MERGED_ANN_FILE, 'w') as outfile:
    json.dump(coco_anns, outfile)

/data/productgraph_from-hive_non-lifestyle/
/data/productgraph_from-hive_non-lifestyle/Bookcases
/data/productgraph_from-hive_non-lifestyle/Nightstands
/data/productgraph_from-hive_non-lifestyle/Stools
/data/productgraph_from-hive_non-lifestyle/Television_Stands
/data/productgraph_from-hive_non-lifestyle/Dressers
/data/productgraph_from-hive_non-lifestyle/Area_Rugs
/data/productgraph_from-hive_non-lifestyle/Desks
/data/productgraph_from-hive_non-lifestyle/Lamps


In [13]:
with open(MERGED_ANN_FILE, 'w') as outfile:
    json.dump(coco_anns, outfile)

In [4]:
SRC_VAL_ANN_FILE = '/data/COCO_2017/annotations/instances_val2017_og.json'
MERGED_VAL_ANN_FILE = '/data/COCO_2017/annotations/instances_val2017.json'
coco_anns_val = json.load(open(SRC_VAL_ANN_FILE))
coco_anns_val['categories'] = coco_anns_val['categories'] + categories
with open(MERGED_VAL_ANN_FILE, 'w') as outfile:
    json.dump(coco_anns_val, outfile)